In [7]:
import torch
import torch.nn as nn
import numpy as np
import pickle
import re
from PIL import Image
import matplotlib.pyplot as plt
import pandas as pd


class VAE(nn.Module):
    def __init__(self, latent_dim=28, image_size=(200, 200)):
        super(VAE, self).__init__()
        self.latent_dim = latent_dim
        self.image_size = image_size

        self.conv1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=4, stride=2, padding=1)
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=4, stride=2, padding=1)
        self.conv3 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=4, stride=2, padding=1)
        
        self.flattened_size = self.calculateFlattenedSize(image_size)
        
        self.fc1 = nn.Linear(self.flattened_size, 512)
        self.fc21 = nn.Linear(512, self.latent_dim)
        self.fc22 = nn.Linear(512, self.latent_dim)

        self.fc3 = nn.Linear(self.latent_dim, 512)
        self.fc4 = nn.Linear(512, self.flattened_size)
        
        self.deconv1 = nn.ConvTranspose2d(128, 64, kernel_size=4, stride=2, padding=1)
        self.deconv2 = nn.ConvTranspose2d(64, 32, kernel_size=4, stride=2, padding=1)
        self.deconv3 = nn.ConvTranspose2d(32, 3, kernel_size=4, stride=2, padding=1)

        self.bn1 = nn.BatchNorm2d(32)
        self.bn2 = nn.BatchNorm2d(64)
        self.bn3 = nn.BatchNorm2d(128)

    def calculateFlattenedSize(self, shape):
        x = torch.zeros(1, 3, *shape)
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        return int(torch.prod(torch.tensor(x.size())))

    def encoder(self, x, sentiment_vector):
        x = nn.functional.relu(self.bn1(self.conv1(x)))
        x = nn.functional.relu(self.bn2(self.conv2(x)))
        x = nn.functional.relu(self.bn3(self.conv3(x)))
        x = x.view(x.size(0), -1)
        h1 = nn.functional.relu(self.fc1(x))
        mu = self.fc21(h1) + sentiment_vector  
        logvar = self.fc22(h1)
        return mu, logvar

    def decoder(self, z):
        h3 = nn.functional.relu(self.fc3(z))
        h4 = nn.functional.relu(self.fc4(h3))
        batch_size = z.size(0)
        h4 = h4.view(batch_size, 128, 25, 25)

        x = nn.functional.relu(self.deconv1(h4))
        x = nn.functional.relu(self.deconv2(x))
        x = torch.sigmoid(self.deconv3(x))  
        return x

    def forward(self, x, sentiment_vector):
        mu, logvar = self.encoder(x, sentiment_vector)
        z = self.reparameterize(mu, logvar)
        recon_x = self.decoder(z)
        return recon_x, mu, logvar

    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5 * logvar)
        eps = torch.randn_like(std)
        return mu + eps * std

def load_sentiment_model(filepath):
    with open(filepath, 'rb') as f:
        return pickle.load(f)


def preprocess_text(text):
    text = text.lower()
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    stopwords = {'a', 'an', 'and', 'are', 'as', 'at', 'be', 'by', 'for', 'from', 'has',
                 'he', 'in', 'is', 'it', 'its', 'of', 'on', 'that', 'the', 'to', 'was',
                 'were', 'will', 'with'}
    tokens = [word for word in text.split() if word not in stopwords]
    return ' '.join(tokens)


# After fitting, you can now use the pipeline to make predictions on new data
def predict_emotions_with_scores(text, model):
    processed_text = preprocess_text(text)
    # Transform the processed text using the tfidf vectorizer
    X_processed = model.named_steps['tfidf'].transform([processed_text])
    # If a scaler is used, apply it as well
    if 'scaler' in model.named_steps:
        X_processed = model.named_steps['scaler'].transform(X_processed)
    
    # Get the predictions for each emotion
    emotion_scores = model.named_steps['classifier'].predict(X_processed)

    # Convert the predicted values to a vector of emotion scores
    emotion_scores_vector = emotion_scores[0]
    
    return emotion_scores_vector



def main():
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    sentiment_model_path = 'best_sentiment_model.pkl'
    sentiment_model = load_sentiment_model(sentiment_model_path)

    vae_model_path = 'vae_model.pth'
    vae_model = VAE(latent_dim=28).to(device)
    vae_model.load_state_dict(torch.load(vae_model_path, map_location=device))
    vae_model.eval()

    print("\nType 'exit' to quit the program.")
    while True:
        user_input = input("\nEnter a text to analyze emotions: ")
        if user_input.lower() == 'exit':
            print("Exiting...")
            break
        
        emotion_scores_vector = predict_emotions_with_scores(user_input, sentiment_model)

        if np.any(emotion_scores_vector):
            print("\nPredicted emotion vector (sentiment scores):")
            print(emotion_scores_vector)

            sentiment_vector = torch.tensor(emotion_scores_vector, dtype=torch.float32).to(device)

            print(f"Sentiment vector shape: {sentiment_vector.shape}")
            print(f"Sentiment vector: {sentiment_vector}")

            try:
                with torch.no_grad():
                    generated_image = vae_model.decoder(sentiment_vector.unsqueeze(0))  
                    generated_image = generated_image.squeeze(0).cpu().numpy().transpose(1, 2, 0)

                print(f"Generated image shape: {generated_image.shape}")

                plt.imshow(generated_image)
                plt.axis('off')
                plt.show()

            except Exception as e:
                print(f"Error in image generation: {e}")
        else:
            print("No significant emotions detected.")



if __name__ == "__main__":
    main()


UnpicklingError: invalid load key, '\x0e'.